# EFM Cosmology: Inflationary Analogue Simulation (α = 1.0) - v2

This notebook simulates the EFM inflationary analogue phase, driven by high-energy dynamics characterized by the state parameter α = 1.0, using EFM principles.

## EFM Theoretical Grounding:
1.  **Scalar Field (φ):** Evolution of the fundamental Eholokon field.
2.  **NLKG Equation (from 'Fluxonic Cosmology...' paper, Eq.1):
    `∂²φ/∂t² - c²∇²φ + m²φ + gφ³ + ηφ⁵ - αφ(∂φ/∂t)·∇φ - δ(∂φ/∂t)²φ = 8πGkφ²`
    For this epoch, α = 1.0. The term ` αφ(∂φ/∂t)·∇φ` is set to 0 pending EFM clarification of its scalar form.
    The term `+ δ(∂φ/∂t)²φ` (when moved to RHS) is considered a potential driving term for expansion.
3.  **Inflationary Dynamics:** Expansion is expected to be driven by the interplay of the potential (m², g, η terms), the δ-term, and self-gravity at high field values.
4.  **Observables:** Total field energy, effective scale factor, and expansion rate H_EFM.

## Objective:
- Demonstrate if exponential energy/scale factor growth occurs from initial noise when α = 1.0 under these EFM assumptions.
- Observe the behavior of the expansion rate H_EFM.

In [ ]:
import os
import torch
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
import gc
import psutil
import torch.nn.functional as F
import torch.amp
from tqdm import tqdm

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print(f"System RAM: {psutil.virtual_memory().total / 1e9:.2f} GB")

output_path_cosmo_inf = os.path.expanduser('~/EFM_Simulations/Cosmology/Inflation_v2/')
os.makedirs(output_path_cosmo_inf, exist_ok=True)
print(f"Cosmology Inflation outputs will be saved to: {output_path_cosmo_inf}")

## Configuration for Inflationary Analogue Simulation (v2)

In [ ]:
config_inf = {}
config_inf['dims'] = 2 # Start with 2D for quicker tests
config_inf['N'] = 256 
config_inf['L_sim'] = 1.0 
config_inf['dx_sim'] = config_inf['L_sim'] / config_inf['N']

config_inf['c_sim'] = 1.0
config_inf['m_sq_sim'] = 0.25     # As per paper's general list for cosmological EOM
config_inf['g_sim'] = 2.0
config_inf['eta_sim'] = 0.01 
config_inf['alpha_sim_coeff'] = 1.0  # This is the coefficient for the 'alpha_term_contribution'
config_inf['delta_sim'] = 0.05     # Coefficient for the δ(∂φ/∂t)²φ term
config_inf['k_sim_gravity'] = 0.01
config_inf['G_sim_gravity'] = 1.0  

config_inf['dt_cfl_factor'] = 1e-5 # Significantly reduced for stability test
config_inf['dt_sim'] = config_inf['dt_cfl_factor'] * config_inf['dx_sim'] / config_inf['c_sim']
config_inf['T_steps'] = 2000

config_inf['initial_phi_amplitude'] = 1e-4
config_inf['run_id'] = f"EFM_Cosmo_Inf_v2_N{config_inf['N']}_alpha{config_inf['alpha_sim_coeff']}_{config_inf['dims']}D_dt{config_inf['dt_sim']:.1e}"
config_inf['history_every_n_steps'] = 20 # More frequent history for debugging
config_inf['checkpoint_every_n_steps'] = 500

print(f"--- EFM Inflation Simulation V2 Configuration ({config_inf['run_id']}) ---")
for key, value in config_inf.items():
    if isinstance(value, (float, np.float32, np.float64)):
        print(f"{key}: {value:.4g}")
    else:
        print(f"{key}: {value}")
print(f"Simulation time step dt_sim: {config_inf['dt_sim']:.4e}")

## Core Simulation Functions for EFM Cosmology (Inflation - v2)

In [ ]:
DEBUG_STEP_COUNT_INF = 0
PRINT_DEBUG_UPTO_STEP = 5 # Print detailed debug info for the first N steps

def get_laplacian_efm(phi_grid, dx_val, dims):
    phi_grid_f32 = phi_grid.to(torch.float32).unsqueeze(0).unsqueeze(0)
    if dims == 1:
        stencil_np = np.array([1, -2, 1], dtype=np.float32) / (dx_val**2)
        stencil = torch.from_numpy(stencil_np).view(1, 1, -1).to(phi_grid.device)
        phi_padded = F.pad(phi_grid_f32, (1,1), mode='circular')
        laplacian = F.conv1d(phi_padded, stencil, padding=0)
    elif dims == 2:
        stencil_np = np.array([[0,1,0],[1,-4,1],[0,1,0]], dtype=np.float32) / (dx_val**2)
        stencil = torch.from_numpy(stencil_np).view(1, 1, 3, 3).to(phi_grid.device)
        phi_padded = F.pad(phi_grid_f32, (1,1,1,1), mode='circular')
        laplacian = F.conv2d(phi_padded, stencil, padding=0)
    elif dims == 3:
        stencil_np = np.array([[[0,0,0],[0,1,0],[0,0,0]],
                               [[0,1,0],[1,-6,1],[0,1,0]],
                               [[0,0,0],[0,1,0],[0,0,0]]], dtype=np.float32) / (dx_val**2)
        stencil = torch.from_numpy(stencil_np).view(1, 1, 3, 3, 3).to(phi_grid.device)
        phi_padded = F.pad(phi_grid_f32, (1,1,1,1,1,1), mode='circular')
        laplacian = F.conv3d(phi_padded, stencil, padding=0)
    else:
        raise ValueError("dims must be 1, 2, or 3")
    return laplacian.squeeze(0).squeeze(0).to(phi_grid.dtype)

def alpha_term_efm(alpha_coeff, phi_curr, phi_dot_curr, grad_phi_list_curr):
    """
    Calculates the EFM alpha term: `α_coeff * φ * (∂φ/∂t) · ∇φ`.
    ASSUMPTION: For scalar φ, interpreting '· ∇φ' as needing a specific scalar EFM form.
    Given the EFM context, this term might represent interaction with an implicit background
    or a higher-order self-interaction. Without explicit scalar derivation from EFM compendium,
    THIS IS SET TO ZERO to avoid introducing unstable/undefined operations.
    If a non-zero form is intended, it must be derived from EFM first principles.
    """
    global DEBUG_STEP_COUNT_INF, PRINT_DEBUG_UPTO_STEP
    if DEBUG_STEP_COUNT_INF < PRINT_DEBUG_UPTO_STEP and alpha_coeff != 0.0:
        print(f"Debug (Call {DEBUG_STEP_COUNT_INF}): alpha_term_efm called. alpha_coeff={alpha_coeff}. Currently returning 0 due to ambiguity.")
    # return alpha_coeff * phi_curr * phi_dot_curr * dot_product_gradient(grad_phi_list_curr) # Example of a *possible* scalar interpretation (highly speculative)
    return torch.zeros_like(phi_curr) # Safest: set to zero until EFM scalar form is clarified

def nlkg_efm_cosmo_rhs_v2(phi_curr, phi_dot_curr, cfg):
    global DEBUG_STEP_COUNT_INF, PRINT_DEBUG_UPTO_STEP
    lap_phi = get_laplacian_efm(phi_curr, cfg['dx_sim'], cfg['dims'])
    grad_phi_list = get_gradient(phi_curr, cfg['dx_sim'], cfg['dims']) # Needed if alpha term is non-zero

    comp_lap = cfg['c_sim']**2 * lap_phi
    comp_m_sq = -cfg['m_sq_sim'] * phi_curr
    comp_g = -cfg['g_sim'] * torch.pow(phi_curr, 3)
    comp_eta = -cfg['eta_sim'] * torch.pow(phi_curr, 5)
    comp_alpha = alpha_term_efm(cfg['alpha_sim_coeff'], phi_curr, phi_dot_curr, grad_phi_list)
    comp_delta = cfg['delta_sim'] * torch.pow(phi_dot_curr, 2) * phi_curr # This is +δ(...) on RHS
    comp_gravity = cfg['G_sim_gravity'] * cfg['k_sim_gravity'] * 8.0 * float(np.pi) * torch.pow(phi_curr, 2)

    phi_ddot_val = comp_lap + comp_m_sq + comp_g + comp_eta + comp_alpha + comp_delta + comp_gravity

    if DEBUG_STEP_COUNT_INF < PRINT_DEBUG_UPTO_STEP:
        print(f"-- RHS Call {DEBUG_STEP_COUNT_INF} --")
        print(f"  phi range: [{torch.min(phi_curr):.2e}, {torch.max(phi_curr):.2e}]")
        print(f"  phi_dot range: [{torch.min(phi_dot_curr):.2e}, {torch.max(phi_dot_curr):.2e}]")
        print(f"  comp_lap range: [{torch.min(comp_lap):.2e}, {torch.max(comp_lap):.2e}] (Max abs: {torch.max(torch.abs(comp_lap)):.2e})")
        print(f"  comp_m_sq range: [{torch.min(comp_m_sq):.2e}, {torch.max(comp_m_sq):.2e}] (Max abs: {torch.max(torch.abs(comp_m_sq)):.2e})")
        print(f"  comp_g range: [{torch.min(comp_g):.2e}, {torch.max(comp_g):.2e}] (Max abs: {torch.max(torch.abs(comp_g)):.2e})")
        print(f"  comp_eta range: [{torch.min(comp_eta):.2e}, {torch.max(comp_eta):.2e}] (Max abs: {torch.max(torch.abs(comp_eta)):.2e})")
        print(f"  comp_alpha range: [{torch.min(comp_alpha):.2e}, {torch.max(comp_alpha):.2e}] (Max abs: {torch.max(torch.abs(comp_alpha)):.2e})")
        print(f"  comp_delta range: [{torch.min(comp_delta):.2e}, {torch.max(comp_delta):.2e}] (Max abs: {torch.max(torch.abs(comp_delta)):.2e})")
        print(f"  comp_gravity range: [{torch.min(comp_gravity):.2e}, {torch.max(comp_gravity):.2e}] (Max abs: {torch.max(torch.abs(comp_gravity)):.2e})")
        print(f"  phi_ddot_val range: [{torch.min(phi_ddot_val):.2e}, {torch.max(phi_ddot_val):.2e}] (Max abs: {torch.max(torch.abs(phi_ddot_val)):.2e})")
        if torch.any(torch.isnan(phi_ddot_val)) or torch.any(torch.isinf(phi_ddot_val)):
            print("  ERROR: NaN/Inf in phi_ddot_val!")
    del grad_phi_list # cleanup
    return phi_ddot_val

def update_phi_rk4_efm_cosmo_v2(phi, phi_dot, dt_val, cfg):
    global DEBUG_STEP_COUNT_INF
    # Autocast for the entire RK4 step, if desired, or within nlkg_efm_cosmo_rhs_v2
    # Using float32 for primary computation as per previous instability
    with torch.amp.autocast(device_type=device.type, dtype=torch.float32, enabled=False): # Disable autocast for debugging precision
        DEBUG_STEP_COUNT_INF = 0 # Reset for k1, k2, k3, k4 calls for this step
        k1_a = nlkg_efm_cosmo_rhs_v2(phi, phi_dot, cfg)
        k1_v = phi_dot

        phi_temp_k2 = phi + 0.5 * dt_val * k1_v
        phi_dot_temp_k2 = phi_dot + 0.5 * dt_val * k1_a
        DEBUG_STEP_COUNT_INF +=1
        k2_a = nlkg_efm_cosmo_rhs_v2(phi_temp_k2, phi_dot_temp_k2, cfg)
        k2_v = phi_dot_temp_k2

        phi_temp_k3 = phi + 0.5 * dt_val * k2_v
        phi_dot_temp_k3 = phi_dot + 0.5 * dt_val * k2_a
        DEBUG_STEP_COUNT_INF +=1
        k3_a = nlkg_efm_cosmo_rhs_v2(phi_temp_k3, phi_dot_temp_k3, cfg)
        k3_v = phi_dot_temp_k3

        phi_temp_k4 = phi + dt_val * k3_v
        phi_dot_temp_k4 = phi_dot + dt_val * k3_a
        DEBUG_STEP_COUNT_INF +=1
        k4_a = nlkg_efm_cosmo_rhs_v2(phi_temp_k4, phi_dot_temp_k4, cfg)
        k4_v = phi_dot_temp_k4

        phi_next = phi + (dt_val / 6.0) * (k1_v + 2*k2_v + 2*k3_v + k4_v)
        phi_dot_next = phi_dot + (dt_val / 6.0) * (k1_a + 2*k2_a + 2*k3_a + k4_a)
    
    del k1_v, k1_a, k2_v, k2_a, k3_v, k3_a, k4_v, k4_a
    del phi_temp_k2, phi_dot_temp_k2, phi_temp_k3, phi_dot_temp_k3, phi_temp_k4, phi_dot_temp_k4
    return phi_next, phi_dot_next

# Energy computation and other utilities (get_effective_scale_factor, print_once) remain the same as your previous version
# but ensure compute_efm_energy_cosmo uses dot_product_gradient for consistency if that's the intended energy.
def dot_product_gradient_efm(grad_list):
    if not grad_list: return torch.tensor(0.0, device=grad_list[0].device if grad_list else device)
    sum_sq = torch.zeros_like(grad_list[0])
    for comp in grad_list:
        sum_sq += comp**2
    return sum_sq

def compute_efm_energy_cosmo_v2(phi, phi_dot, cfg):
    phi_f32 = phi.to(torch.float32)
    phi_dot_f32 = phi_dot.to(torch.float32)
    vol_element = cfg['dx_sim']**cfg['dims']
    kinetic_density = 0.5 * torch.pow(phi_dot_f32, 2)
    potential_density_V = 0.5 * cfg['m_sq_sim'] * torch.pow(phi_f32, 2) + \
                          0.25 * cfg['g_sim'] * torch.pow(phi_f32, 4) + \
                          (1.0/6.0) * cfg['eta_sim'] * torch.pow(phi_f32, 6)
    grad_phi_list_eng = get_gradient(phi_f32, cfg['dx_sim'], cfg['dims'])
    grad_sq_density = 0.5 * cfg['c_sim']**2 * dot_product_gradient_efm(grad_phi_list_eng)
    total_energy = torch.sum(kinetic_density + grad_sq_density + potential_density_V) * vol_element
    del phi_f32, phi_dot_f32, grad_phi_list_eng # Cleanup
    return total_energy.item()

def get_effective_scale_factor_efm(phi_sq_sum, initial_phi_sq_sum, dims):
    if initial_phi_sq_sum < 1e-30: return 1.0 
    if phi_sq_sum < 1e-30: return float('inf') 
    return (initial_phi_sq_sum / phi_sq_sum)**(1.0/dims)

def print_once_efm(msg, rank_to_print=0):
    print(msg)


## Main Simulation Loop for Inflationary Analogue (v2)

In [ ]:
print(f"Starting EFM Inflationary Analogue Simulation v2: {config_inf['run_id']}")
DEBUG_STEP_COUNT_INF = 0 # Reset global debug counter

np.random.seed(42)
phi_dims = tuple([config_inf['N']] * config_inf['dims'])
phi_np_init = np.random.randn(*phi_dims).astype(np.float32) * config_inf['initial_phi_amplitude']

phi = torch.from_numpy(phi_np_init).to(device, dtype=torch.float32) # Using float32
phi_dot = torch.zeros_like(phi, dtype=torch.float32, device=device)
del phi_np_init
gc.collect()
print(f"Fields initialized on {device}. Phi shape: {phi.shape}, dtype: {phi.dtype}")

num_hist_points_inf = config_inf['T_steps'] // config_inf['history_every_n_steps'] + 1
sim_time_history = np.zeros(num_hist_points_inf, dtype=np.float64)
energy_history = np.zeros(num_hist_points_inf, dtype=np.float64)
phi_sq_sum_history = np.zeros(num_hist_points_inf, dtype=np.float64)
scale_factor_history = np.zeros(num_hist_points_inf, dtype=np.float64)
h_efm_history = np.zeros(num_hist_points_inf, dtype=np.float64)
hist_idx_inf = 0
current_time_sim = 0.0

initial_phi_sq_sum = torch.sum(phi**2).item()
sim_time_history[hist_idx_inf] = current_time_sim
energy_history[hist_idx_inf] = compute_efm_energy_cosmo_v2(phi, phi_dot, config_inf)
phi_sq_sum_history[hist_idx_inf] = initial_phi_sq_sum
scale_factor_history[hist_idx_inf] = 1.0
h_efm_history[hist_idx_inf] = 0.0
print(f"Step 0: Time={current_time_sim:.2e}, Energy={energy_history[hist_idx_inf]:.3e}, PhiSqSum={phi_sq_sum_history[hist_idx_inf]:.3e}")
hist_idx_inf += 1

sim_start_time_loop = time.time()
numerical_error_occurred_inf = False
pbar_inflation = tqdm(range(config_inf['T_steps']), desc=f"EFM Inflation ({config_inf['run_id']})")

for t_step in pbar_inflation:
    DEBUG_STEP_COUNT_INF = 0 # Reset for each RK4 step's internal calls
    try:
        if torch.any(torch.isinf(phi)) or torch.any(torch.isnan(phi)) or \
           torch.any(torch.isinf(phi_dot)) or torch.any(torch.isnan(phi_dot)):
            print(f"\nERROR: NaN/Inf in fields BEFORE step {t_step + 1}! Stopping.")
            numerical_error_occurred_inf = True; break

        phi, phi_dot = update_phi_rk4_efm_cosmo_v2(phi, phi_dot, config_inf['dt_sim'], config_inf)
        current_time_sim += config_inf['dt_sim']

        if torch.any(torch.isinf(phi)) or torch.any(torch.isnan(phi)):
            print(f"\nERROR: NaN/Inf in phi AFTER step {t_step + 1}! Stopping.")
            numerical_error_occurred_inf = True; break

        if (t_step + 1) % config_inf['history_every_n_steps'] == 0:
            if hist_idx_inf < num_hist_points_inf:
                current_energy_val = compute_efm_energy_cosmo_v2(phi, phi_dot, config_inf)
                current_phi_sq_sum_val = torch.sum(phi**2).item()
                current_scale_factor_val = get_effective_scale_factor_efm(current_phi_sq_sum_val, initial_phi_sq_sum, config_inf['dims'])

                sim_time_history[hist_idx_inf] = current_time_sim
                energy_history[hist_idx_inf] = current_energy_val
                phi_sq_sum_history[hist_idx_inf] = current_phi_sq_sum_val
                scale_factor_history[hist_idx_inf] = current_scale_factor_val
                
                if hist_idx_inf > 0 and scale_factor_history[hist_idx_inf-1] > 1e-9 and current_scale_factor_val > 1e-9:
                    log_a_prev_val = np.log(scale_factor_history[hist_idx_inf-1])
                    log_a_curr_val = np.log(current_scale_factor_val)
                    dt_hist_val = sim_time_history[hist_idx_inf] - sim_time_history[hist_idx_inf-1]
                    if dt_hist_val > 1e-12: 
                         h_efm_history[hist_idx_inf] = (log_a_curr_val - log_a_prev_val) / dt_hist_val
                
                pbar_inflation.set_postfix({'E': f'{current_energy_val:.2e}', 'a_eff': f'{current_scale_factor_val:.2e}', 'H': f'{h_efm_history[hist_idx_inf]:.2e}'})
                if np.isnan(current_energy_val) or np.isinf(current_energy_val):
                    print(f"Instability: Energy NaN/Inf at step {t_step+1}. Stop.")
                    numerical_error_occurred_inf = True; break
                hist_idx_inf += 1
    except Exception as e_loop_outer_inf:
        print(f"ERROR in Inflation sim loop at step {t_step + 1}: {e_loop_outer_inf}")
        import traceback; traceback.print_exc()
        numerical_error_occurred_inf = True; break

pbar_inflation.close()
sim_duration_total_inf = time.time() - sim_start_time_loop
print(f"EFM Inflation simulation loop finished in {sim_duration_total_inf:.2f} s. Error: {numerical_error_occurred_inf}")

if not numerical_error_occurred_inf and hist_idx_inf < num_hist_points_inf:
    sim_time_history[hist_idx_inf] = current_time_sim
    energy_history[hist_idx_inf] = compute_efm_energy_cosmo_v2(phi, phi_dot, config_inf)
    current_phi_sq_sum_val = torch.sum(phi**2).item()
    phi_sq_sum_history[hist_idx_inf] = current_phi_sq_sum_val
    scale_factor_history[hist_idx_inf] = get_effective_scale_factor_efm(current_phi_sq_sum_val, initial_phi_sq_sum, config_inf['dims'])
    if hist_idx_inf > 0 and scale_factor_history[hist_idx_inf-1] > 1e-9 and scale_factor_history[hist_idx_inf] > 1e-9:
        log_a_prev_val = np.log(scale_factor_history[hist_idx_inf-1])
        log_a_curr_val = np.log(scale_factor_history[hist_idx_inf])
        dt_hist_val = sim_time_history[hist_idx_inf] - sim_time_history[hist_idx_inf-1]
        if dt_hist_val > 1e-12: h_efm_history[hist_idx_inf] = (log_a_curr_val - log_a_prev_val) / dt_hist_val
    hist_idx_inf += 1

actual_hist_len_inf = hist_idx_inf
if actual_hist_len_inf > 1:
    plt.figure(figsize=(18, 5))
    plt.subplot(1, 3, 1)
    plt.plot(sim_time_history[:actual_hist_len_inf], energy_history[:actual_hist_len_inf])
    plt.xlabel('Time (sim units)'); plt.ylabel('Total Energy (sim units)')
    plt.title('Energy vs. Time (Inflation)'); plt.grid(True); plt.yscale('log')

    plt.subplot(1, 3, 2)
    plt.plot(sim_time_history[:actual_hist_len_inf], scale_factor_history[:actual_hist_len_inf])
    plt.xlabel('Time (sim units)'); plt.ylabel('Effective Scale Factor a_eff')
    plt.title('Scale Factor vs. Time (Inflation)'); plt.grid(True); plt.yscale('log')

    plt.subplot(1, 3, 3)
    # Plot H_EFM only where it's meaningful (after the first history point)
    valid_h_efm_indices = np.where(sim_time_history[:actual_hist_len_inf][1:] > 0)[0] + 1
    if len(valid_h_efm_indices) > 0:
        plt.plot(sim_time_history[valid_h_efm_indices], h_efm_history[valid_h_efm_indices])
    plt.xlabel('Time (sim units)'); plt.ylabel('H_EFM (sim units)')
    plt.title('Expansion Rate H_EFM vs. Time (Inflation)'); plt.grid(True)
    
    plt.tight_layout()
    plt.suptitle(f"EFM Inflationary Analogue ({config_inf['run_id']})", fontsize=16, y=1.03)
    plt.savefig(os.path.join(output_path_cosmo_inf, f"{config_inf['run_id']}_evolution.png"))
    plt.show()
else:
    print("Not enough history points to plot inflation results.")

del phi, phi_dot
gc.collect()
if torch.cuda.is_available(): torch.cuda.empty_cache()
print("Inflation simulation cell finished.")